# demo 1

Here we will do the following:  
- load a nifty file
- generate a fixed brain and use it for motion correction
- cluster the pixels in each volume's slice
- save the pixel-labels 

07 dec 2023  
sama ahmed

In [1]:
import numpy as np
import h5py
from scipy.ndimage import gaussian_filter1d

from nre import io, moco, roi
import ants

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## helper functions

In [2]:
def _zscore(x):
    """x: 2D array (n, t)"""

    x_mean = np.mean(x, axis=-1)
    x_std = np.std(x, axis=-1)
    return (x - x_mean[:, None]) / x_std[:, None]

def _zdff(F, win=200, smooth=False):
    """calculate zscored(df/f) based on F baseline activity"""

    # find average signal in first `win` volumes
    Fbase = np.mean(F[:, :win], axis=-1)
    dff = (F - Fbase[:, None]) / Fbase[:, None]

    if smooth:
        dff = gaussian_filter1d(dff, sigma=1)
    
    return _zscore(dff)


## hyperparams 

## set paths

In [3]:
nifty_path = 'data/TSeries-12012023-1437-028_channel_1.nii'
labels_path = 'data/demo_labels.h5'  # for saving the cluster assignments

## load data

In [4]:
# load xyzt
brain = io.load(nifty_path)

n_slices = brain.shape[2]  # z
n_vols = brain.shape[-1]  # t

## generate fixed brain

This is typically made from the structural channel (e.g. red channel, tdtomato) but also fine to make it from the green, GCaMP channel. You simply average across time to get an xyz volume. If you have a long recording session, it's totally fine to use the first couple of minutes (or "x" volumetric timepoints) to generate the "fixed mean brain". e.g.  

```
# mean_brain based on the first 300 volumes
mean_brain = np.mean(brain[0:300], axis=-1)
fixed = ants.from_numpy(mean_brain)
```

In [5]:
mean_brain = np.mean(brain, axis=-1)
fixed = ants.from_numpy(mean_brain)

### save the fixed brain as .nii if you want

Note that `io.save` requires a numpy array so we'll use the `mean_brain` variable

In [6]:
# io.save('data/fixed.nii', mean_brain)

## motion correction

for each volume (aka timepoint), warp the volume to the fixed template using SyN transformation

In [7]:
moco_brain = np.zeros_like(brain)

for vol in range(n_vols):
    moving = ants.from_numpy(brain[:, :, :, vol])
    moco_brain[:, :, :, vol] = moco.apply(fixed, moving).numpy()

## extract ROIs

There are lots of ways to extract ROIs. A simple approach is to cluster neighboring pixels in a brain slice based on their correlated activity patterns. Here we use hierarchical clustering to specify the clusters. This is essentially dimensionality reduction.  

This approach is based on [Brezovec et al 2022](https://www.biorxiv.org/content/10.1101/2022.03.20.485047v1)


In [8]:
n_clusters = 100  # this can be any number... I suggest 500 or 1000. Brezovec et al used 2000 

### generate clusters

This can take a while...   
One way to speed it up is by downsampling in time

```
ds = moco_brain[:, :, iSlice, 0:-1:5]  # every 5th timepoint
cluster_model = roi.create_2d_clusters(ds, n_clusters, 'dat/cluster_mem')    
```

In [9]:
labels = []

# for each slice in Z, generate n_clusters of pixels and return the pixel label
for iSlice in range(moco_brain.shape[2]):
    cluster_model = roi.create_2d_clusters(moco_brain[:, :, iSlice, :], n_clusters, 'tmp/cluster_mem')    
    labels.append(cluster_model.labels_)

### save the cluster labels. 

In [10]:
hf = h5py.File(labels_path, 'w')
hf.create_dataset('labels', data=labels)
hf.close()

## get zscored df/f for each cluster in each slice

### set the baseline F window

set the window we'll use to calculate the baseline fluorescence  

Here it's set `n_vols` because our demo dataset is small. Depending on your experiment, `F_WINDOW` could be encompass the first couple of minutes of activity before stimulus turns on

Alternatively, the `F_WINDOW` could be a few seconds before each stimulus, in which case you'll need to amend this code 

In [11]:
F_WINDOW = n_vols

In [12]:
ROIs = np.empty((n_slices, n_clusters, n_vols))

for iSlice in range(n_slices):
    mean_signal = np.empty(shape=(n_vols, n_clusters))

    for vol in range(n_vols):
        mean_supervox, _ = roi.get_supervoxel_mean_2D(moco_brain[:, :, iSlice, vol], labels[iSlice], n_clusters)
        mean_signal[vol] = mean_supervox

    # find zscored(df/f) and smooth over time
    ROIs[iSlice, :, :] = _zdff(mean_signal.T, win=F_WINDOW, smooth=True)

## save the zscored(df/f) values for all ROIs

This has shape: n_slices, n_clusters, n_vols

In [13]:
io.save('data/ROIs.nii', ROIs)